---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [15]:
def graph_identification():
#     G = P1_Graphs[1]
    
#     degrees = dict(G.degree())
#     degree_values = sorted(set(degrees.values()))
#     hist = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    
#     plt.bar(degree_values, hist)
#     plt.xlabel('Degree')
#     plt.ylabel('Fraction of nodes')
#     plt.show()
    
    
#     plt.plot(degree_values, hist, 'o')
#     plt.xlabel('Degree')
#     plt.ylabel('Fraction of nodes')
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
#graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [16]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def salary_predictions():
    
    
    df = pd.DataFrame(G.nodes())
    feats = ['clustering', 'degree', 'closeness', 'betweenness', 'degree_centrality', 'page_rank']
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    
    def check_for_nan(n):
        n = n[1]['ManagementSalary']
        if n == 0:
            return 0
        elif n == 1:
            return 1
        else:
            return None
    
    df['management'] = pd.Series([check_for_nan(node) for node in G.nodes(data=True)])
    
    scaler = MinMaxScaler()
    train = df[~pd.isnull(df['management'])]
    test = df[pd.isnull(df['management'])]
    
    x_train = train[feats]
    x_train_scaled = scaler.fit_transform(x_train)
    y_train = train['management']
    
    x_test = test[feats]
    x_test_scaled = scaler.fit_transform(x_test)
    
    
    clf = MLPClassifier(hidden_layer_sizes = [5, 8, 5], alpha = 0.2, random_state = 0, solver='sgd', learning_rate_init = 0.02)
    clf.fit(x_train_scaled, y_train)
    y_test = clf.predict_proba(x_test_scaled)[:, 1]
    
    res = pd.Series(y_test, x_test.index)
    
    return res
#salary_predictions()

1       0.461667
2       0.979402
5       0.999969
8       0.364233
14      0.881544
18      0.655693
27      0.784865
30      0.845666
31      0.531357
34      0.361583
37      0.285852
40      0.666042
45      0.519013
54      0.676694
55      0.640505
60      0.603356
62      0.999997
65      0.978996
77      0.168772
79      0.393999
97      0.229461
101     0.139788
103     0.627603
108     0.379790
113     0.947052
122     0.106558
141     0.932001
142     0.998767
144     0.123920
145     0.902691
          ...   
913     0.097312
914     0.172463
915     0.038064
918     0.341913
923     0.056170
926     0.168937
931     0.105274
934     0.038064
939     0.038064
944     0.038064
945     0.058566
947     0.291453
950     0.312162
951     0.065561
953     0.047075
959     0.038064
962     0.038064
963     0.409659
968     0.181261
969     0.161220
974     0.148743
984     0.038064
987     0.196464
989     0.185068
991     0.188170
992     0.038064
994     0.038064
996     0.0380

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [35]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [46]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def new_connections_predictions():
    
    scaler = MinMaxScaler()
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    
    pref_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index = [(x[0], x[1]) for x in pref_attachment])
    df['preferential_attachment'] = [x[2] for x in pref_attachment]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
#     cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
#     df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df['cn_soundarajan_hopcroft'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
#     df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df = future_connections.join(df,how='outer')
    train = df[~pd.isnull(df['Future Connection'])]
    test = df[pd.isnull(df['Future Connection'])]
    feats = ['preferential_attachment', 'jaccard_coefficient', 'resource_allocation_index', 'cn_soundarajan_hopcroft']
    x_train = train[feats]
    x_train_scaled = scaler.fit_transform(x_train)
    y_train = train['Future Connection']
    x_test = test[feats]
    x_test_scaled = scaler.fit_transform(x_test)
    clf = MLPClassifier(hidden_layer_sizes = [5, 8, 5], alpha = 0.2, random_state = 0, solver='sgd', learning_rate_init = 0.02)
    clf.fit(x_train_scaled, y_train)
    y_test = clf.predict_proba(x_test_scaled)[:, 1]
    predictions = pd.Series(y_test, x_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    
    return target['prob']
#new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.014881
(542, 751)    0.009681
(20, 426)     0.717404
(50, 989)     0.009679
(942, 986)    0.009676
(324, 857)    0.009679
(13, 710)     0.182310
(19, 271)     0.168784
(319, 878)    0.009678
(659, 707)    0.009681
(49, 843)     0.009677
(208, 893)    0.009680
(377, 469)    0.009785
(405, 999)    0.011478
(129, 740)    0.011073
(292, 618)    0.014656
(239, 689)    0.009678
(359, 373)    0.009740
(53, 523)     0.044679
(276, 984)    0.009677
(202, 997)    0.009677
(604, 619)    0.131238
(270, 911)    0.009678
(261, 481)    0.059860
(200, 450)    0.989655
(213, 634)    0.009682
(644, 735)    0.063801
(346, 553)    0.009688
(521, 738)    0.009700
(422, 953)    0.012510
                ...   
(672, 848)    0.009678
(28, 127)     0.970168
(202, 661)    0.009689
(54, 195)     0.996992
(295, 864)    0.009679
(814, 936)    0.009681
(839, 874)    0.009676
(139, 843)    0.009679
(461, 544)    0.009708
(68, 487)     0.009710
(622, 932)    0.009680
(504, 936)    0.012143
(479, 528) 